## Van Gogh - se

## Configuração


### Carregando bibliotecas

In [0]:
import tensorflow as tf
import tensorflow_hub as hub

In [0]:
import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import PIL.Image
import time
import functools

from os import listdir
from os.path import isfile, join
import random

### Baixando imagens base para teste da rede

In [0]:
!bash -c 'rm -rf /content/*'
!bash -c 'mkdir /content/vangogh'
!bash -c 'mkdir /content/vangogh_results'
!bash -c 'mkdir /content/photos'
!wget -O /content/photos/example.jpeg https://github.com/FabioRick/vangogh-se/releases/download/dataset-v0.1/example.jpeg
!wget -O /content/images.zip https://github.com/FabioRick/vangogh-se/releases/download/dataset-v0.1/images.zip
!unzip /content/images.zip -d /content/vangogh
!bash -c 'rm -rf /content/images.zip'

### Definindo funções utilitárias

In [0]:
def tensor_to_image(tensor):
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)

def load_img(path_to_img):
  max_dim = 512
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img

def imshow(image, title=None):
  if len(image.shape) > 3:
    image = tf.squeeze(image, axis=0)

  plt.imshow(image)
  if title:
    plt.title(title)

## Selecionando e visualizando os dados de entrada

In [0]:
content_dir_path = "/content/photos"
content_path = random.choice([ join(content_dir_path, f) for f in listdir(content_dir_path) if isfile(join(content_dir_path, f))])
style_dir_path = "/content/vangogh"
style_path = random.choice([ join(style_dir_path, f) for f in listdir(style_dir_path) if isfile(join(style_dir_path, f))])

content_image = load_img(content_path)
style_image = load_img(style_path)

plt.subplot(1, 2, 1)
imshow(content_image, 'Content Image')

plt.subplot(1, 2, 2)
imshow(style_image, 'Style Image')

## Carregando e executando modelo

### Carregando modelo já treinado para transferência de

In [0]:
hub_module = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

Rodando o modelo e visualizando o resultado

In [0]:
stylized_image = hub_module(tf.constant(content_image), tf.constant(style_image))[0]
tensor_to_image(stylized_image)